# Quick Tour

## Preparations

The following cell loads the AiiDA Jupyter extensions. This enables interacting with the database and daemon, displaying node graphs and more. If you want to run this notebook yourself, you must have AiiDA and the `aiida-c2sm` plugin installed (`pip install [-e] .` from the project root), computer and codes set up and run at least one `GCM2Icon` workflow successfully. You will need to change the PK vari

In [65]:
%load_ext aiida
%aiida

The aiida extension is already loaded. To reload it, use:
  %reload_ext aiida


Loaded AiiDA DB environment - profile name: c2sm.

Next we import the modules we need from AiiDA, our plugin and some pretty printing utilities.

In [66]:
from aiida import orm, plugins
from aiida.engine import processes
import aiida_c2sm
import tabulate

## Interacting with the database programatically

the `aiida.orm` module contains everything needed to talk to the database. One example is the powerful `QueryBuilder`, which allows to build very complex graph queries. Here we only use it in it's simplest form to display the successfully finished runs.

In [113]:
q = orm.QueryBuilder()
q.append(aiida_c2sm.spice.gcm2icon.workflow.Gcm2Icon)
q.all()
successful_runs = [run[0] for run in q.all() if run[0].process_state is processes.ProcessState.FINISHED]

print(tabulate.tabulate([(run.pk, run.inputs.experiment_id.value) for run in successful_runs], headers=["PK", "experiment ID"]))

/Users/ricoh/.virtualenvs/aiida-c2sm/lib/python3.10/site-packages/aiida/orm/querybuilder.py:1385: AiidaEntryPointWarning: Process type 'aiida_c2sm.spice.gcm2icon.workflow.Gcm2Icon' does not correspond to a registered entry. This risks queries to fail once the location of the process class changes. Add an entry point for 'aiida_c2sm.spice.gcm2icon.workflow.Gcm2Icon' to remove this warning.
  warnings.warn(


  PK  experiment ID
----  -----------------
3251  poetic-lorikeet
3188  remarkable-raptor
3355  maroon-porcupine
3420  impetuous-manatee
3634  hot-clam


Next, we pick one of the runs and look at all the AiiDA-processes it started. It is stored in the database as a `WorkChainNode` (`WorkChain` being the class in AiiDA to represent a workflow). Some of these might be `CalcJobs` which were submitted to the cluster, some might be sub-`WorkChain`s. This is an example of things that are stored with the run.

In [112]:
PK = 3634
node = orm.load_node(PK)
print(
    tabulate.tabulate(
        [(c.pk, c.label.split(":")[0], c.label.split("@")[-1], c.ctime, c.mtime, c.process_state, c.__class__.__name__, c.outputs.remote_folder.get_remote_path()) for c in node.called],
        headers=["PK", "Stage", "Experiment start time", "Process creation time", "Process modification time", "Process state", "Database node type", "Run folder on the cluster"]
    )
)

  PK  Stage    Experiment start time      Process creation time             Process modification time         Process state          Database node type    Run folder on the cluster
----  -------  -------------------------  --------------------------------  --------------------------------  ---------------------  --------------------  -------------------------------------------------------------------
3636  prep     1979-01-01T00:00:00+00:00  2023-08-22 15:08:12.293136+02:00  2023-08-22 15:21:02.558395+02:00  ProcessState.FINISHED  CalcJobNode           /scratch/snx3000/ricoh/aiida/78/b1/80ce-5c16-49b2-b88b-1c990602dc18
3641  conv     1979-01-01T00:00:00+00:00  2023-08-22 15:21:02.649733+02:00  2023-08-22 15:26:04.341467+02:00  ProcessState.FINISHED  CalcJobNode           /scratch/snx3000/ricoh/aiida/ea/59/7ff4-1302-423d-8916-8756f189d65a
3652  icon     1979-01-01T00:00:00+00:00  2023-08-22 15:26:04.589979+02:00  2023-08-22 15:40:24.409404+02:00  ProcessState.FINISHED  WorkChainNode    

## Interacting with the database via CLI

We can print out information about nodes via the `verdi` commandline. For example, we can see that the icon run above is a workflow. Let's look at what `verdi` can tell us about it.

In [83]:
!verdi process show 3468

Property     Value
-----------  ----------------------------------------------------------------------------
type         IconWorkChain
state        Finished [11] The process did not register a required output.
pk           3468
uuid         25acc5e7-d3b5-4231-9758-a573d172af67
label        icon:impetuous-manatee@1979-02-01T00:00:00+00:00
description  Icon run for experiment impetuous-manatee,  launched from Gcm2Icon workflow.
ctime        2023-08-22 11:50:36.959490+02:00
mtime        2023-08-22 11:54:10.143648+02:00

Inputs                 PK  Type
-------------------  ----  -------------
boundary_data        1964  RemoteData
code                 2514  InstalledCode
ecraddir             2084  RemoteData
expid                3419  Str
extpar_relpath       3465  Str
gcm_converted        3460  RemoteData
ghg_file_relpath     3466  Str
ini_basedir          2079  RemoteData
inidata              1964  RemoteData
lam_grid_relpath     3463  Str
options              3462  Dict
parameters      

We can see that it called an Icon `CalcJob` with the PK 3476. Let's look at the scheduler output for that one

In [84]:
!verdi process report 3476

*** 3476: None
*** Scheduler output:


Batch Job Summary Report (version 21.01.1) for Job "aiida-3476" (48486102) on daint

Job information (1/2)
-----------------------------------------------------------------------------------------------------
             Submit            Eligible               Start                 End    Elapsed Time limit
------------------- ------------------- ------------------- ------------------- ---------- ----------
2023-08-22T11:52:01 2023-08-22T11:52:01 2023-08-22T11:52:37 2023-08-22T11:53:07   00:00:30   01:00:00
-----------------------------------------------------------------------------------------------------

Job information (2/2)
-------------------------------------------------------------
    Username      Account    Partition   NNodes        Energy
------------ ------------ ------------ -------- -------------
       ricoh      csstaff       normal       10     22.773 kJ


*** Scheduler errors:
Switching to atp/3.14.5.
Switching to cray-libsci

The `verdi` commandline has a generic `node show` command as well as different `show` commands for some types of nodes. Compare the output of above `process show` with the `code show` below.

In [137]:
prep_code = orm.load_code("spice-prep-installed")
prep_code.pk

1978

In [102]:
!verdi code show 1978

-----------------------  ------------------------------------
PK                       1978
UUID                     579759a5-2a6b-4970-9bcd-366a7a50f993
Type                     core.code.installed
Computer                 Daint-mpirun (daint.cscs.ch), pk: 1
Filepath executable      /users/ricoh/spicebin/prep.sh
Label                    spice-prep-installed
Description              Pre-Installed spice prep script.
Default calc job plugin  c2sm.spice_prep
Use double quotes        False
With mpi
Prepend text             module load daint-gpu
                         module load gcc/9.3.0
                         module load CDO/2.0.5-CrayGNU-21.09
                         module load NCO/5.0.4-CrayGNU-21.09
Append text
-----------------------  ------------------------------------
